In [1]:
#第14章/虚拟数据
import torch

#假设有2句话，8个词
x = torch.ones(2, 8)

#两句话中各有一些pad
x[0, 6:] = 0
x[1, 7:] = 0

x

tensor([[1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.]])

In [2]:
#第14章/定义key_padding_mask
#key_padding_mask的定义方式，就是x中是pad的为True，否则是False
key_padding_mask = x == 0

key_padding_mask

tensor([[False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False,  True]])

In [3]:
#第14章/定义encode_attn_mask
#在encode阶段不需要定义encode_attn_mask
#定义为None或者全False都可以
encode_attn_mask = torch.ones(8, 8) == 0

encode_attn_mask

tensor([[False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]])

In [4]:
#第14章/定义decode_attn_mask
#在decode阶段需要定义decode_attn_mask
#decode_attn_mask的定义方式是对角线以上为True的上三角矩阵
decode_attn_mask = torch.tril(torch.ones(8, 8)) == 0

decode_attn_mask

tensor([[False,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False]])

In [5]:
#第14章/编码x
x = x.unsqueeze(2)
x = x.expand(-1, -1, 12)

x, x.shape

(tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 torch.Size([2, 8, 12]))

In [6]:
#第14章/定义multi_head_attention_forward()所需要的参数

#in_proj就是qkv线性变换的参数
in_proj_weight = torch.nn.Parameter(torch.randn(3 * 12, 12))
in_proj_bias = torch.nn.Parameter(torch.zeros((3 * 12)))

#out_proj就是输出时做线性变换的参数
out_proj_weight = torch.nn.Parameter(torch.randn(12, 12))
out_proj_bias = torch.nn.Parameter(torch.zeros(12))

in_proj_weight.shape, in_proj_bias.shape, out_proj_weight.shape, out_proj_bias.shape

(torch.Size([36, 12]),
 torch.Size([36]),
 torch.Size([12, 12]),
 torch.Size([12]))

In [7]:
#第14章/使用工具函数计算多头注意力
data = {
    #因为不是batch_first的，所以需要变形一下
    'query': x.permute(1, 0, 2),
    'key': x.permute(1, 0, 2),
    'value': x.permute(1, 0, 2),
    'embed_dim_to_check': 12,
    'num_heads': 2,
    'in_proj_weight': in_proj_weight,
    'in_proj_bias': in_proj_bias,
    'bias_k': None,
    'bias_v': None,
    'add_zero_attn': False,
    'dropout_p': 0.2,
    'out_proj_weight': out_proj_weight,
    'out_proj_bias': out_proj_bias,
    'key_padding_mask': key_padding_mask,
    'attn_mask': encode_attn_mask,
}

score, attn = torch.nn.functional.multi_head_attention_forward(**data)

score.shape, attn, attn.shape

(torch.Size([8, 2, 12]),
 tensor([[[0.2083, 0.1042, 0.2083, 0.2083, 0.2083, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.1042, 0.2083, 0.2083, 0.1042, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.2083, 0.1042, 0.1042, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.2083, 0.1042, 0.1042, 0.2083, 0.0000, 0.0000],
          [0.1042, 0.2083, 0.2083, 0.2083, 0.1042, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.2083, 0.2083, 0.2083, 0.1042, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.1042, 0.1042, 0.2083, 0.1042, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.2083, 0.2083, 0.2083, 0.2083, 0.0000, 0.0000]],
 
         [[0.1786, 0.1786, 0.1786, 0.0893, 0.0893, 0.0893, 0.1786, 0.0000],
          [0.1786, 0.0893, 0.0893, 0.0893, 0.1786, 0.1786, 0.1786, 0.0000],
          [0.0893, 0.1786, 0.1786, 0.1786, 0.1786, 0.1786, 0.1786, 0.0000],
          [0.0893, 0.1786, 0.1786, 0.0893, 0.0893, 0.0000, 0.0893, 0.0000],
          [0.1786, 0.0893, 0.0893, 0.0893, 0.1786, 0.0000, 0

In [8]:
#第14章/使用多头注意力工具层
multihead_attention = torch.nn.MultiheadAttention(embed_dim=12,
                                                  num_heads=2,
                                                  dropout=0.2,
                                                  batch_first=True)

data = {
    'query': x,
    'key': x,
    'value': x,
    'key_padding_mask': key_padding_mask,
    'attn_mask': encode_attn_mask,
}

score, attn = multihead_attention(**data)

score.shape, attn, attn.shape

(torch.Size([2, 8, 12]),
 tensor([[[0.2083, 0.2083, 0.2083, 0.2083, 0.2083, 0.1042, 0.0000, 0.0000],
          [0.2083, 0.1042, 0.2083, 0.2083, 0.0000, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.2083, 0.1042, 0.1042, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.0000, 0.2083, 0.2083, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.1042, 0.1042, 0.0000, 0.2083, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.1042, 0.2083, 0.2083, 0.1042, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.1042, 0.2083, 0.2083, 0.2083, 0.2083, 0.0000, 0.0000],
          [0.2083, 0.2083, 0.2083, 0.1042, 0.2083, 0.2083, 0.0000, 0.0000]],
 
         [[0.1786, 0.0893, 0.0893, 0.1786, 0.0893, 0.1786, 0.1786, 0.0000],
          [0.1786, 0.1786, 0.1786, 0.1786, 0.1786, 0.1786, 0.1786, 0.0000],
          [0.1786, 0.0893, 0.0893, 0.0893, 0.0893, 0.1786, 0.0893, 0.0000],
          [0.0893, 0.0893, 0.0893, 0.0893, 0.0893, 0.0893, 0.1786, 0.0000],
          [0.1786, 0.1786, 0.0893, 0.1786, 0.1786, 0.1786, 0

In [9]:
#第14章/使用单层编码器工具层
encoder_layer = torch.nn.TransformerEncoderLayer(
    d_model=12,
    nhead=2,
    dim_feedforward=24,
    dropout=0.2,
    activation=torch.nn.functional.relu,
    batch_first=True,
    norm_first=True)

data = {
    'src': x,
    'src_mask': encode_attn_mask,
    'src_key_padding_mask': key_padding_mask,
}

out = encoder_layer(**data)

out.shape

torch.Size([2, 8, 12])

In [10]:
#第14章/使用编码器工具层
encoder = torch.nn.TransformerEncoder(
    encoder_layer=encoder_layer,
    num_layers=3,
    norm=torch.nn.LayerNorm(normalized_shape=12))

data = {
    'src': x,
    'mask': encode_attn_mask,
    'src_key_padding_mask': key_padding_mask,
}

out = encoder(**data)

out.shape

torch.Size([2, 8, 12])

In [11]:
#第14章/使用单层解码器工具层
decoder_layer = torch.nn.TransformerDecoderLayer(
    d_model=12,
    nhead=2,
    dim_feedforward=24,
    dropout=0.2,
    activation=torch.nn.functional.relu,
    batch_first=True,
    norm_first=True)

data = {
    'tgt': x,
    'memory': x,
    'tgt_mask': decode_attn_mask,
    'memory_mask': encode_attn_mask,
    'tgt_key_padding_mask': key_padding_mask,
    'memory_key_padding_mask': key_padding_mask,
}

out = decoder_layer(**data)

out.shape

torch.Size([2, 8, 12])

In [12]:
#第14章/使用编码器工具层
decoder = torch.nn.TransformerDecoder(
    decoder_layer=decoder_layer,
    num_layers=3,
    norm=torch.nn.LayerNorm(normalized_shape=12))

data = {
    'tgt': x,
    'memory': x,
    'tgt_mask': decode_attn_mask,
    'memory_mask': encode_attn_mask,
    'tgt_key_padding_mask': key_padding_mask,
    'memory_key_padding_mask': key_padding_mask,
}

out = decoder(**data)

out.shape

torch.Size([2, 8, 12])

In [15]:
#第14章/使用Transformer工具模型
transformer = torch.nn.Transformer(d_model=12,
                                   nhead=2,
                                   num_encoder_layers=3,
                                   num_decoder_layers=3,
                                   dim_feedforward=24,
                                   dropout=0.2,
                                   activation=torch.nn.functional.relu,
                                   custom_encoder=encoder,
                                   custom_decoder=decoder,
                                   batch_first=True,
                                   norm_first=True)

data = {
    'src': x,
    'tgt': x,
    'src_mask': encode_attn_mask,
    'tgt_mask': decode_attn_mask,
    'memory_mask': encode_attn_mask,
    'src_key_padding_mask': key_padding_mask,
    'tgt_key_padding_mask': key_padding_mask,
    'memory_key_padding_mask': key_padding_mask,
}

out = transformer(**data)

out.shape

torch.Size([2, 8, 12])